## Содержание

- Содержание
- Цель проекта
- Подготовка данных
- Модели, качество, время обучения
- Тестированые лучшей модели
- Итог

## Цель проекта

</br> Заказщик нуждается в модели машинного обучения которая может предсказать цену автомобиля.
</br> Нам предаставленны данные описывающие машини и их цены
</br> Модель должна быть точной и быстро обучаться

#### Библиотеки

In [68]:
RANDOM = 2022
FIRST_CAR_YEAR = 1886 # Benz Patent Motor Car
YEAR = 2022
MASS_PRODUCED_CAR_MAX_HP = 1914 # Rimac Nevera

import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score

import lightgbm as lgb

## Подготовка данных

### Загружаем данные

In [2]:
localPath = './datasets/autos.csv'
webPath = '/datasets/autos.csv'

path = ''
if (os.path.exists(localPath)):
  path = localPath
elif (os.path.exists(webPath)):
  path = webPath

df = pd.read_csv(path)

### Данные которые не пригодятся для обучения модели

In [3]:
irrelevant_cols = ['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen']

In [4]:
df = df.drop(irrelevant_cols, axis=1)

### Проверка типов данных

In [5]:
df.dtypes

Price                 int64
VehicleType          object
RegistrationYear      int64
Gearbox              object
Power                 int64
Model                object
Kilometer             int64
RegistrationMonth     int64
FuelType             object
Brand                object
Repaired             object
dtype: object

In [6]:
num_cols = ['Price', 'RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']
cat_cols = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

### Подготовка колличественных данных

In [7]:
display(df[num_cols].describe())

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645
std,4514.158514,90.227958,189.850405,37905.341530,3.726421
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


- 'Price' не может быть 0 -> заменить на NaN
- 'RegistrationYear' должен быть в диапозоне [1886, 2023] -> остальные, заменить на NaN
- 'Power' долшен быть в диапозоне (0, 1479] -> остальные, заменить на NaN
- 'Kilometer' ок
- 'RegistrationMonth' ок

In [8]:
df_num = df[num_cols]

In [9]:
df_num['Price'] = df_num['Price'].replace(0, None)

In [10]:
def yearRangeFilter(x):
    if x < FIRST_CAR_YEAR or x > YEAR:
        return None
    return x

In [11]:
df_num['RegistrationYear'] = df_num['RegistrationYear'].apply(lambda x: yearRangeFilter(x))

In [12]:
def powerRangeFilter(x):
    if x <= 0 or x > MASS_PRODUCED_CAR_MAX_HP:
        return None
    return x

In [13]:
df_num['Power'] = df_num['Power'].apply(lambda x: powerRangeFilter(x))

In [14]:
display(df_num.describe())

,RegistrationYear,Power,Kilometer,RegistrationMonth
count,354198.000000,314005.000000,354369.000000,354369.000000
mean,2003.084789,120.971061,128211.172535,5.714645
std,7.536418,62.642107,37905.341530,3.726421
min,1910.000000,1.000000,5000.000000,0.000000
25%,1999.000000,75.000000,125000.000000,3.000000
50%,2003.000000,110.000000,150000.000000,6.000000
75%,2008.000000,150.000000,150000.000000,9.000000
max,2019.000000,1910.000000,150000.000000,12.000000


In [15]:
df[num_cols] = df_num

In [16]:
del df_num

По пропускам решим что делать позже

### Очистка категоральных данных

In [17]:
df_cat = df[cat_cols]

In [18]:
for c in df_cat.columns:
    print(c, len(df_cat[c].unique()))

VehicleType 9
Gearbox 3
Model 251
FuelType 8
Brand 40
Repaired 3


In [19]:
display(df_cat['VehicleType'].value_counts())

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: VehicleType, dtype: int64

- Не видно ошибок, данные распределенны хорошо
- Категория 'other' не пригодится -> заменит 'other' на 'None'

In [20]:
df_cat['VehicleType'] = df_cat['VehicleType'].replace('other', None)

In [21]:
display(df_cat['Gearbox'].value_counts())

manual    268251
auto       66285
Name: Gearbox, dtype: int64

Не видно ошибок, данные распределенны хорошо

In [22]:
display(df_cat['FuelType'].value_counts())

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

- Gasoline и petrol это одно и тоже -> заменить 'gasoline' на 'petrol'
- Категория 'other' не пригодится -> заменить 'other' на 'None'

In [23]:
df_cat['FuelType'] = df_cat['FuelType'].replace('gasoline', 'petrol')

In [24]:
df_cat['FuelType'] = df_cat['FuelType'].replace('other', None)

In [25]:
display(df_cat['Repaired'].value_counts())

no     247161
yes     36054
Name: Repaired, dtype: int64

Ошибок не наблюдается

In [26]:
print(sorted(df_cat['Brand'].unique()))

['alfa_romeo', 'audi', 'bmw', 'chevrolet', 'chrysler', 'citroen', 'dacia', 'daewoo', 'daihatsu', 'fiat', 'ford', 'honda', 'hyundai', 'jaguar', 'jeep', 'kia', 'lada', 'lancia', 'land_rover', 'mazda', 'mercedes_benz', 'mini', 'mitsubishi', 'nissan', 'opel', 'peugeot', 'porsche', 'renault', 'rover', 'saab', 'seat', 'skoda', 'smart', 'sonstige_autos', 'subaru', 'suzuki', 'toyota', 'trabant', 'volkswagen', 'volvo']


Ошибки не наблюдается

In [27]:
print(df_cat['Model'].unique()[:100])

['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz']


Ошибки не наблюдается

In [28]:
df[cat_cols] = df_cat

In [29]:
del df_cat

### Анализ пропусков

In [30]:
n_rows_with_missing_data = df.isna().any(axis=1).sum()

In [31]:
percentage_of_rows_missing_data = n_rows_with_missing_data / len(df.index) * 100

In [32]:
print('percentage_of_rows_missing_data %.2f' % percentage_of_rows_missing_data)

percentage_of_rows_missing_data 34.62


In [33]:
df_miss = df.isna()

In [34]:
def get_miss_count(row):
    out = 0
    for x in row:
        if x:
            out += 1
    return out

In [35]:
miss_count = df_miss.apply(lambda row: get_miss_count(row), axis=1)

In [36]:
print(miss_count.value_counts())

0    231699
1     62239
2     32086
3     15008
4      6026
5      4290
6      2279
7       723
8        19
dtype: int64


Что делать с пропусками:
- Дропнуть строки с ними
- Заменить средним или медианой
- Заменить на часто встречающиеся данные
- Самые близкие данные k-NN
<br>
<br> Мое предпочтение по тому что делать с пропусками
1. Дропнуть их
2. Дропнуть строки с двумя или более пропусками на строку, остальные запольнить каким нибудь способом
3. Запольнить каким нибудь способом

### Пропуски

In [37]:
miss_percent = (len(df.index) - len(df.dropna().index)) / len(df.index) * 100

In [38]:
print('%', 'of missing values', '%.2f' % miss_percent)

% of missing values 34.62


In [39]:
df = df.dropna()

### Преобразование категоральных данных

Как их можно преобразовать
- Мало категорий
  - OHE
- Много категорий
  - Label Encoding - сработает если в категориях есть иерархия
  - Frequency Encoding - проблема возникнет если две категории встречаются одинаково часто
  - Mean Target Encoding - найти средние показание целевого признака и заменит им

И много других методов...

In [1]:
print(cat_cols)

NameError: name 'cat_cols' is not defined

In [ ]:
df_cat = df[cat_cols]

In [ ]:
for c in df_cat:
    print(c, len(df_cat[c].unique()))

VehicleType 7
Gearbox 2
Model 248
FuelType 5
Brand 39
Repaired 2


OHE для всех, кроме Model - Median Target Encoding (подойдет лучше всего)

In [43]:
df_cat = df_cat.join(pd.get_dummies(df_cat['VehicleType']))
df_cat = df_cat.join(pd.get_dummies(df_cat['Gearbox']))
df_cat = df_cat.join(pd.get_dummies(df_cat['FuelType']))
df_cat = df_cat.join(pd.get_dummies(df_cat['Brand']))
df_cat = df_cat.join(pd.get_dummies(df_cat['Repaired']))

In [44]:
models = df_cat['Model'].unique()

In [45]:
models_median_target = {}

In [46]:
for m in models:
    targets = df['Price'][df_cat['Model'] == m]
    median = targets.median()
    models_median_target[m] = median

In [47]:
df_cat['Model_target'] = df_cat['Model'].apply(lambda x: models_median_target[x])

In [48]:
print(df_cat.columns)

Index(['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired',
       'bus', 'convertible', 'coupe', 'sedan', 'small', 'suv', 'wagon', 'auto',
       'manual', 'cng', 'electric', 'hybrid', 'lpg', 'petrol', 'alfa_romeo',
       'audi', 'bmw', 'chevrolet', 'chrysler', 'citroen', 'dacia', 'daewoo',
       'daihatsu', 'fiat', 'ford', 'honda', 'hyundai', 'jaguar', 'jeep', 'kia',
       'lada', 'lancia', 'land_rover', 'mazda', 'mercedes_benz', 'mini',
       'mitsubishi', 'nissan', 'opel', 'peugeot', 'porsche', 'renault',
       'rover', 'saab', 'seat', 'skoda', 'smart', 'subaru', 'suzuki', 'toyota',
       'trabant', 'volkswagen', 'volvo', 'no', 'yes', 'Model_target'],
      dtype='object')


Теперь удалим cat_cols и вернем к df, удалим cat_cols из df

In [49]:
df_cat = df_cat.drop(cat_cols, axis=1)

In [50]:
df = df.join(df_cat)

In [51]:
df = df.drop(cat_cols, axis=1)

In [52]:
print(df.shape)

(231699, 61)


### Маштабирование

In [53]:
df_target = df['Price']

In [54]:
df = df.drop('Price', axis=1)

In [56]:
scaler = MinMaxScaler()
scaler.fit(df)
df[df.columns] = scaler.transform(df)

In [58]:
df.head(3)

,RegistrationYear,Power,Kilometer,RegistrationMonth,bus,convertible,coupe,sedan,small,suv,...,smart,subaru,suzuki,toyota,trabant,volkswagen,volvo,no,yes,Model_target
3,0.842593,0.038968,1.000000,0.500000,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.161123
4,0.907407,0.035808,0.586207,0.583333,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.192548
5,0.787037,0.053186,1.000000,0.833333,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.203734


In [59]:
df = df.join(df_target)

### Train Valid Test

In [73]:
df_x = df.drop('Price', axis=1)
df_y = df['Price']

In [74]:
df_x_train_valid, df_x_test, df_y_train_valid, df_y_test = \
    train_test_split(df_x, df_y, test_size=0.2, random_state=RANDOM)

In [75]:
df_x_train, df_x_valid, df_y_train, df_y_valid = \
    train_test_split(df_x_train_valid, df_y_train_valid, test_size=0.25, random_state=RANDOM)

## Models | parameters | time - scores

#### Linear Regression

In [76]:
model = LinearRegression().fit(df_x_train.values, df_y_train.values)

In [77]:
df_a_train = model.predict(df_x_train.values)

In [78]:
r2_score(df_y_train, df_a_train)

0.6603066275692331

0.66 не плохой показатель

In [79]:
df_a_valid = model.predict(df_x_valid.values)

In [80]:
r2_score(df_y_valid, df_a_valid)

0.6686758802355859

0.69 еще лучше чем на валидационных данных чем на тренировачных! :)

#### LightGBM

In [99]:
g_x_train = df_x_train
g_y_train = df_y_train
g_x_valid = df_x_valid
g_y_valid = df_y_valid

In [100]:
for c in g_x_train:
    g_x_train[c] = g_x_train[c].astype('float')

g_y_train = g_y_train.astype('float')

for c in g_x_valid:
    g_x_valid[c] = g_x_valid[c].astype('float')

g_y_valid = g_y_valid.astype('float')

In [101]:
lgb_train = lgb.Dataset(g_x_train, g_y_train)
lgb_valid = lgb.Dataset(g_x_valid, g_y_valid, reference=lgb_train)

In [102]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [103]:
# print('Starting training...')
# gbm = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=20,
#                 valid_sets=lgb_valid,
#                 callbacks=[lgb.early_stopping(stopping_rounds=5)])

# print('Saving model...')
# gbm.save_model('model.txt')

Я не могу понять почемы у меня не получается тренировать модель, какая-то проблемма с данными

## Тестирование лучшей модели

## Итог

Надеюсь все проделанные шаги логичны, жду помощи на мой вопрос с тренировкой LGB